R# Preprocesamiento de los comentarios para la primera fase de fine-tuning y entrenamiento del modelo
En esta primera fase, una vez extraídos los bodies de todos los issues/PRs de los repositorios listados junto con 2 comentarios de 250 del total de issues de cada repositorio, se prepararán todos los datos para ser utilizados en el entrenamiento y ajuste fino del modelo que se utilizará.

Se seguirán prácticamente los mismos pasos vistos en *c_preparing_data_for_statistics_and_ML* pero con varias diferencias claves que existen entre los modelos BERT que se utilizarán ahora y los modelos de clasificación presentados con anterioridad (notebooks de GVTIA).

Para Transformers funciona mejor un preprocesado mínimo y dejar la segmentación al propio tokenizador del modelo, a continuación se muestra que procedimientos similares a los anteriores se mantendrán y cuáles se evitarán:

## Se mantendrá:
- Normalización de espacios/saltos de línea
- Eliminación de caracteres de control raros o poco usuales
- Se conservará el uso de mayúsculas y minúsculas, signos, números, URLs, nombres propios de vulnerabilidades o bugs (CVE-XXXX-YYYY), rutas (/etc/...), código entre backticks (`return salida`), nombres de APIs.
- Se definirá una longitud máxima de tokens por comentario o el uso de *sliding window* si el texto es muy largo.

## Se omitirá:
- Pasar todo el texto a minúsculas, los modelos RoBERTa/DistilRoBERTa que se utilizarán utilizan mayúsculas y minúsculas.
- Eliminar la puntuación y stopwords.
- Stemming / lematización.
- Normalizaciones agresivas de URLs/código -> se pierde señal técnica.

Una vez explicado esto, se comenzará con el preprocesado de todos los comentarios extraídos de GitHub, comenzando como se ha visto ya en diversas ocasiones, con cargar el documento (.csv) en un dataframe de pandas para su uso y manipulación.

En este caso, se cuenta con 2 documentos:
- **gh_bodys_lastyear.csv**. Archivo que contiene los bodies (comentario principal) de todos los Issues/PRs en el último año de los repositorios listados para la extracción de comentarios.
- **gh_comments_lastyear.csv**. Archivo que contiene los 2 primeros comentarios de cada Issue/PR de 250 Issues/PRs por repositorio (500 comentarios por repo), en gran parte de los casos serán las respuestas aportadas por usuarios al body del documento anterior.

En este caso, como se cuenta con 2 documentos lo que se hará es crear 2 dataframes, uno con cada documento, para a continuación unirlos con la función `concat()` de pandas y ordenarlos según el id del Issue/PR para la clara visualización y mantener una estructura coherente entre cuerpo principal y comentarios asociados.

In [1]:
import pandas as pd

# Ruta de los archivos
path_gh_bodys = "../data/gh_comments/train-fine_tuning/gh_bodys_lastyear.csv"
path_gh_comments = "../data/gh_comments/train-fine_tuning/gh_comments_lastyear.csv"

# Carga de los archivos en DataFrames
df_bodys = pd.read_csv(path_gh_bodys)
df_comms = pd.read_csv(path_gh_comments)

In [2]:
print(df_bodys.columns)
print(df_comms.columns)

Index(['repo', 'is_pr', 'issue_number', 'comment_type', 'comment_id',
       'comment_created_at', 'comment_author', 'text', 'comment_url',
       'context_id', 'container_title', 'container_state', 'container_url',
       'container_created_at', 'container_updated_at', 'container_labels'],
      dtype='object')
Index(['kubernetes/kubernetes', 'False', '133680', 'issue_comment',
       'github_issuecomment_IC_kwDOAToIks6_4TOW', '2025-08-25T07:51:17Z',
       'k8s-ci-robot',
       'This issue is currently awaiting triage.\nIf a SIG or subproject determines this is a relevant issue, they will accept it by applying the triage/accepted label and provide further guidance.\nThe triage/accepted label can be added by org members by writing /triage accepted in a comment.\n\nInstructions for interacting with me using PR comments are available here.  If you have questions or suggestions related to my behavior, please file an issue against the kubernetes-sigs/prow repository.',
       'https://gi

Se ha cometido un error en la escritura de la cabecera de los comentarios por escribir siempre en el mismo documento y borrar su contenido en vez de eliminar el documento antes de comenzar con una nueva extracción. Vamos a tratar de repararlo sin tener que volver a realizar todo el proceso de extracción.

In [3]:
from pathlib import Path
path_gh_bodys = Path(path_gh_bodys)
path_gh_comments = Path(path_gh_comments)

EXPECTED_COLS = [
    'repo','is_pr','issue_number','comment_type','comment_id','comment_created_at','comment_author',
    'text','comment_url','context_id','container_title','container_state','container_url',
    'container_created_at','container_updated_at','container_labels'
]

def read_with_header_fix(p: Path) -> pd.DataFrame:
    # Se lee 1 fila para inspeccionar columnas
    probe = pd.read_csv(p, nrows=1)
    if list(probe.columns) == EXPECTED_COLS:
        return pd.read_csv(p)
    # Si no coincide, reinterpretamos: no hay cabecera -> header=None + names=EXPECTED_COLS
    return pd.read_csv(p, header=None, names=EXPECTED_COLS)

df_bodys = read_with_header_fix(path_gh_bodys)
df_comms = read_with_header_fix(path_gh_comments)

# Se unen ambos DataFrames
df_gh = pd.concat([df_bodys, df_comms], ignore_index=True)

# Tipos y ordenación
df_gh['comment_created_at'] = pd.to_datetime(df_gh['comment_created_at'], errors='coerce', utc=True)
df_gh.loc[df_gh['comment_created_at'].isna(), 'comment_created_at'] = pd.to_datetime(df_gh['container_created_at'], errors='coerce', utc=True)

order_map = {'issue_body':0, 'pr_body':0} # Bodies primero -> coherencia
df_gh['order'] = df_gh['comment_type'].map(order_map).fillna(1).astype(int)

df_gh = df_gh.sort_values(by=['repo','issue_number','order','comment_created_at','comment_id'], kind='mergesort').drop(columns=['order'])

# Normalizar booleano -> OPCIONAL
df_gh['is_pr'] = df_gh['is_pr'].astype(str).str.lower().map({'true':True, 'false':False})

In [4]:
# Muestra para comprobar que se ha ejecutado correctamente
print(df_bodys.columns)
print(df_comms.columns)

df_gh.head(10).T
print(len(df_gh))

Index(['repo', 'is_pr', 'issue_number', 'comment_type', 'comment_id',
       'comment_created_at', 'comment_author', 'text', 'comment_url',
       'context_id', 'container_title', 'container_state', 'container_url',
       'container_created_at', 'container_updated_at', 'container_labels'],
      dtype='object')
Index(['repo', 'is_pr', 'issue_number', 'comment_type', 'comment_id',
       'comment_created_at', 'comment_author', 'text', 'comment_url',
       'context_id', 'container_title', 'container_state', 'container_url',
       'container_created_at', 'container_updated_at', 'container_labels'],
      dtype='object')
108078


Ahora sí están todos los comentarios bien ordenados. Antes de comenzar con el preprocesado vamos a guardar el dataframe en una base de datos.

In [5]:
print(len(df_gh))

108078


In [6]:
import sqlite3

db_gh = "../data/gh_comments/train-fine_tuning/gh_dataset_lastyear.db"
con = sqlite3.connect(db_gh)
df_gh.to_sql('gh_dataset', con, if_exists='replace', index=False)
con.close()

In [19]:
import pathlib

db_gh = pathlib.Path("../data/gh_comments/train-fine_tuning/gh_dataset_lastyear.db")
with sqlite3.connect(db_gh) as con:
    n_rows = con.execute("SELECT COUNT(*) FROM gh_dataset").fetchone()[0]
print(f"Filas en la tabla: {n_rows:,}")

with sqlite3.connect(db_gh) as con:
    sample = con.execute("""
        SELECT * FROM gh_dataset
        ORDER BY RANDOM()
        LIMIT 10
    """).fetchall()
sample[:3]   # imprime tres filas de ejemplo


Filas en la tabla: 108,078


[('grafana/grafana',
  0,
  105099,
  'issue_body',
  'github_issuebody_grafana/grafana#105099',
  '2025-05-08 10:28:52+00:00',
  'joshhunt',
  'Description\nVerify that the Betterer CI check GitHub Action is functioning correctly in the following release branches:\n\nrelease-11.4.5\nrelease-11.5.5\nrelease-11.3.7\n\nIt should work in all active release branches.\nSee https://github.com/grafana/grafana/actions/runs/14903136485/job/41859445141?pr=104905 for an example.\nI believe this is because we copied the actions from main into all release branches, but the betterer check in main relies on a yarn command which is not present in earlier release branches. We would want to either backport the yarn command, or update the action in the branches.\nSteps to Validate\n\nTrigger the Betterer CI check Github Action for each specified branch.\nEnsure the CI check completes successfully without errors.\n\nExpected Outcome\nThe Betterer CI check should pass for all release branches',
  'https://

Ahora sí se procederá al procesamiento del dataset para dejarlo preparado para el modelo BERT que se utilizará, RoBERTa o DistilRoBERTa. Este proceso se va a definir en una serie de scripts .py, cada uno con el objetivo de realizar una tarea para su reutilización en otros puntos del proyecto (cuando se haga el de reddit, u otros comentarios de github) de forma que estos sean agnósticos al sistema del que se extraen los comentarios que serán utilizados por el modelo.

Del mismo modo, tras el procesamiento de los datos, el resultado del procesado será almacenado en archivos `.parquet` por su ligereza y agilidad a la hora de ser manipulados y consumidos por modelos BERT. Las principales ventajas de este formato son:
- Más pequeño: compresión por columna, pesa de 2 a 5 veces menos que un `.csv`
- Más rápido: lee solo las columnas que se necesitan ("column pruning") y aplica vectorización.
- Conserva tipos: fechas, booleanos, enteros "nullable", etc. (`.csv`los pierde)
- Esquema: guarda el _schema_ dentro del archivo -> menos sorpresas al cargarlo

Por estas características el formato es el preferido para pipelines de datos/ML. En este caso la estructura que se utilizará será:
- `merged.parquet` = será el dataset completo tras ingesta y normalización ligera.
- `split_train.parquet`, `split_dev.parquet`, `split_test.parquet` = particiones del dataset ya divididas, listas para su tokenización.

#### Ventajas frente a CSV
- Evita problemas de comas y saltos de línea
- Mantiene las fechas (`created_at`), booleanos (`is_pr`) y enteros sin perder el tipo.
- Carga solo las columnas necesarias -> menor uso de RAM y tiempo de ejecución.

## Ejecución del preprocesado de datos
Se han definido varios scripts, cada uno con una función en el preprocesado de datos para modelos BERT, a  los cuáles se les llamará desde este notebook con los argumentos correspondientes para realizar este proceso.

Las funciones reutilizables para el pipeline están definidas en `prep_utils.py`. Normaliza texto "ligero", mapea columnas heterogéneas (GH/Reddit) al esquema core (id, text, label, source, created_at, context_id), lee CSV/SQLite y guarda Parquet.
El objetivo principal de estas funciones es su utilización cuando se desea agnosticismo de fuente (Github/Reddit) y un preprocesado mínimo ideal para Transformers.

### `ingest_merge.py`
Combina una o varias entradas en un único DataFrame, deduplica el contenido por `id`, aplica normalización ligera y guarda `merged.parquet` + meta. Se le da como input el archivo `.db` o los `.csv` deseados, produciendo en la salida un DF en formato `.parquet`.

In [7]:
import os
print(os.getcwd())

F:\Mis documentos\CLASE\0. TFG\BERTolto\notebooks_BERTolto


In [8]:
# Define la ruta de tu base de datos tal como la usas
db_gh = "../data/gh_comments/train-fine_tuning/gh_dataset_lastyear.db"

try:
    con = sqlite3.connect(db_gh)
    # Intenta leer algunas filas de la tabla 'gh_dataset'
    test_df = pd.read_sql("SELECT * FROM gh_dataset LIMIT 5", con)
    con.close()

    print(f"\nVerificación de la base de datos '{db_gh}':")
    print(f"Número de filas recuperadas de 'gh_dataset': {len(test_df)}")
    if not test_df.empty:
        print("Primeras filas de 'gh_dataset' (limitadas a 5):")
        print(test_df.head())
    else:
        print("¡Advertencia! La tabla 'gh_dataset' está vacía o no se pudieron recuperar datos.")

except Exception as e:
    print(f"Ocurrió un error al intentar verificar la base de datos: {e}")
    if 'gh_dataset_lastyear.db' not in str(e) and 'no such table' in str(e):
        print("Asegúrate de que la base de datos y la tabla ('gh_dataset') existen y el nombre es correcto.")


Verificación de la base de datos '../data/gh_comments/train-fine_tuning/gh_dataset_lastyear.db':
Número de filas recuperadas de 'gh_dataset': 5
Primeras filas de 'gh_dataset' (limitadas a 5):
                                 repo  is_pr  issue_number   comment_type  \
0  electron-userland/electron-builder      0           690  issue_comment   
1  electron-userland/electron-builder      0          2674  issue_comment   
2  electron-userland/electron-builder      0          3009  issue_comment   
3  electron-userland/electron-builder      0          3009  issue_comment   
4  electron-userland/electron-builder      0          3124  issue_comment   

                                comment_id         comment_created_at  \
0  github_issuecomment_IC_kwDOAiVL48626ksv  2025-07-14 10:14:03+00:00   
1  github_issuecomment_IC_kwDOAiVL4866KmSG  2025-07-26 20:27:33+00:00   
2  github_issuecomment_IC_kwDOAiVL486w_HsW  2025-06-13 07:10:06+00:00   
3  github_issuecomment_IC_kwDOAiVL486x74HL  2025-06-

In [21]:
from src.data_prep.prep_utils import load_sqlite
load_sqlite("../data/gh_comments/train-fine_tuning/gh_dataset_lastyear.db",
            table="gh_dataset",
            limit=10)

,repo,is_pr,issue_number,comment_type,comment_id,comment_created_at,comment_author,text,comment_url,context_id,container_title,container_state,container_url,container_created_at,container_updated_at,container_labels
0,electron-userland/electron-builder,0,690,issue_comment,github_issuecomment_IC_kwDOAiVL48626ksv,2025-07-14 10:14:03+00:00,Wiktor102,Can confirm that specifying the appUrl option ...,https://github.com/electron-userland/electron-...,electron-userland/electron-builder#issue:690,Error: Filename can either be an absolute HTTP...,CLOSED,https://github.com/electron-userland/electron-...,2016-08-22T17:50:08Z,2025-07-14T10:14:03Z,help wanted;investigate;backlog;Squirrel.Windows
1,electron-userland/electron-builder,0,2674,issue_comment,github_issuecomment_IC_kwDOAiVL4866KmSG,2025-07-26 20:27:33+00:00,siikakamania,Ok found problem by elimination. I have folder...,https://github.com/electron-userland/electron-...,electron-userland/electron-builder#issue:2674,makensis.exe exited with code 1,CLOSED,https://github.com/electron-userland/electron-...,2018-03-08T12:31:10Z,2025-07-26T20:27:33Z,None
2,electron-userland/electron-builder,0,3009,issue_comment,github_issuecomment_IC_kwDOAiVL486w_HsW,2025-06-13 07:10:06+00:00,theIYD,Anyone who was able to crack auto-update with ...,https://github.com/electron-userland/electron-...,electron-userland/electron-builder#issue:3009,How to use electron-forge-maker-nsis,CLOSED,https://github.com/electron-userland/electron-...,2018-06-12T16:48:57Z,2025-06-18T18:12:08Z,backlog
3,electron-userland/electron-builder,0,3009,issue_comment,github_issuecomment_IC_kwDOAiVL486x74HL,2025-06-18 18:12:08+00:00,prayash,nice mna,https://github.com/electron-userland/electron-...,electron-userland/electron-builder#issue:3009,How to use electron-forge-maker-nsis,CLOSED,https://github.com/electron-userland/electron-...,2018-06-12T16:48:57Z,2025-06-18T18:12:08Z,backlog
4,electron-userland/electron-builder,0,3124,issue_comment,github_issuecomment_IC_kwDOAiVL486yE1aY,2025-06-19 10:41:44+00:00,minhtan143,This is a necessary feature,https://github.com/electron-userland/electron-...,electron-userland/electron-builder#issue:3124,How to add a custom page/field to NSIS setup,CLOSED,https://github.com/electron-userland/electron-...,2018-07-17T20:19:17Z,2025-06-19T10:41:44Z,backlog
5,electron-userland/electron-builder,0,3185,issue_comment,github_issuecomment_IC_kwDOAiVL4868Kgl1,2025-08-05 22:59:10+00:00,oceangravity,😭,https://github.com/electron-userland/electron-...,electron-userland/electron-builder#issue:3185,Nested node_modules disappears after building,CLOSED,https://github.com/electron-userland/electron-...,2018-07-28T18:22:53Z,2025-08-05T22:59:10Z,None
6,electron-userland/electron-builder,0,3322,issue_comment,github_issuecomment_IC_kwDOAiVL486577nr,2025-07-25 17:00:17+00:00,Pritraj,"It is 2024, what happend on this issue? I hope...",https://github.com/electron-userland/electron-...,electron-userland/electron-builder#issue:3322,Q : Does MSI work with auto-update ?,CLOSED,https://github.com/electron-userland/electron-...,2018-09-14T20:08:28Z,2025-07-25T17:00:17Z,question;msi
7,electron-userland/electron-builder,0,3376,issue_comment,github_issuecomment_IC_kwDOAiVL486d8ofG,2025-02-11 06:33:25+00:00,hanzhenfang,"same to me,",https://github.com/electron-userland/electron-...,electron-userland/electron-builder#issue:3376,Custom path for latest.yml,CLOSED,https://github.com/electron-userland/electron-...,2018-10-10T10:35:13Z,2025-06-19T22:14:59Z,question
8,electron-userland/electron-builder,0,3376,issue_comment,github_issuecomment_IC_kwDOAiVL486yLFqm,2025-06-19 22:14:59+00:00,devPablo,"I'm facing the same issue, need dynamic ""url"" ...",https://github.com/electron-userland/electron-...,electron-userland/electron-builder#issue:3376,Custom path for latest.yml,CLOSED,https://github.com/electron-userland/electron-...,2018-10-10T10:35:13Z,2025-06-19T22:14:59Z,question
9,electron-userland/electron-builder,0,3632,issue_comment,githu

In [23]:
import subprocess, sys

base = "../src/data_prep"

# 1. Ingesta + merge (de CSVs o db en SQLite)
# Como ya tengo el dataset directamente almacenado en un .db
subprocess.run([sys.executable, f"{base}/ingest_merge.py",
                "--sqlite-db", "../data/gh_comments/train-fine_tuning/gh_dataset_lastyear.db",
                "--table", "gh_dataset",
                "--out-parquet", "../src/artifacts/prep/merged.parquet",
                "--out-meta", "../src/artifacts/prep/merged_meta.json"
                ], check=True)


CalledProcessError: Command '['C:\\Users\\diego\\AppData\\Local\\Programs\\Python\\Python310\\python.exe', '../src/data_prep/ingest_merge.py', '--sqlite-db', '../data/gh_comments/train-fine_tuning/gh_dataset_lastyear.db', '--table', 'gh_dataset', '--out-parquet', '../src/artifacts/prep/merged.parquet', '--out-meta', '../src/artifacts/prep/merged_meta.json']' returned non-zero exit status 1.

### `quick_report.py`
Muestra por pantalla un resumen del Parquet aportado en el input: nº de filas, distribución de `label`, distribución de `source`, rango de `created_at`.
Es útil para la verificación visual de que la ingesta es correcta antes del split/tokenización e ideal para detectar desbalanceos fuertes o rangos temporales inesperados antes de la fase de entrenamiento.

In [ ]:
# Informe rápido
subprocess.run([sys.executable, f"{base}/quick_report.py",
                "--in-parquet", "src/artifacts/prep/merged.parquet"
                ], check=True)

### `split_thread_temporal.py`
Divide el DF `merged.parquet` en train/dev/test respetando hilos (`context_id`) y orden temporal para evitar fuga de información entre splits manteniendo el orden cronológico dentro de cada uno.
Los `ratios` son las proporciones con las que se dividirá el dataset:
- train: 70%
- dev: 15%
- test: 15%

In [ ]:
# Split temporal + thread-aware
subprocess.run([sys.executable, f"{base}/split_thread_temporal.py",
                "--in-parquet", "src/artifacts/prep/merged.parquet",
                "--out-train", "src/artifacts/prep/split-train.parquet",
                "--out-dev", "src/artifacts/prep/split-dev.parquet",
                "--out-test", "src/artifacts/prep/split-test.parquet",
                "--ratios", "0.7", "0.15", "0.15",
                "--out-meta", "src/artifacts/prep/split-meta.json"
                ], check=True)

### `tokenize_hf.py`
Carga los Parquet de train/dev/test, antepone un prefijo de dominio (`<GITHUB>`, `<REDDIT>`, en este caso no porque todavía estoy solo con los comentarios de GH), tokeniza los textos con el tokenizador de DistilRoBERTa/RoBERTa y guarda:
- `dataset/` (formato `save_to_disk` de HF Datasets)
- `tokenizer/` (vocabulario + config)
- `preprocess_meta.json` (tamaños, `max_len`, pesos de cada clase)

#### Parámetros
- `--base-model`: modelo que se utilizará, en este caso distilroberta
- `--max-len 384`: longitud de secuencia; bajar acelera, subir captura más contexto.
- `--use-domain-prefix`: etiqueta de la fuente (GITHUB/REDDIT)
- `--sliding-window --slide-stride 128`: para textos muy largos (menos truncado)

Para decidir el `--max-len` y la posibilidad de utilizar `--sliding-window`, se puede calcular la distribución de longitudes en tokens y elegir el valor del argumento para cubrir el *p90-p95* y obtener un mejor resultado.

In [ ]:
from transformers import AutoTokenizer
import pandas as pd

tok = AutoTokenizer.from_pretrained("distilroberta-base", use_fast=True)
df = pd.read_parquet("../src/artifacts/prep/merged.parquet") # o split_train.parquet
lens = df["text"].astype(str).map(lambda s: len(tok(s, truncation=False)["input_ids"]))

print(pd.Series(lens).describe(percentiles=[.5, .9, .95, .99]))

In [ ]:
# tokenización HF (DistilRoBERTa/RoBERTa)
subprocess.run([sys.executable, f"{base}/tokenize_hf.py",
                "--train-parquet", "src/artifacts/prep/split-train.parquet",
                "--dev-parquet", "src/artifacts/prep/split-dev.parquet",
                "--test-parquet", "src/artifacts/prep/split-test.parquet",
                "--out-dir", "src/artifacts/hf_distilroberta",
                "--base-model", "distilroberta-base",
                "--max-len", "384"#,
                #"--use-domain-prefix", # Comentado por ahora que solo hay GH
                ], check=True)